In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import datasets as datasets
import torchvision.transforms as transform
t = transform.Compose([transform.ToTensor(),transform.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=t)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=t)
trainloader = torch.utils.data.DataLoader(train_data, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data)



In [ ]:
#defining fulling connected layer 
class FClayer:
        def __init__(self,Sin,Sout):
            self.Sin = Sin
            self.Sout = Sout
            self.W = np.random.randn(Sin,Sout)/np.sqrt(Sin)    # initializing weights and bias matrices with random normal distribition 
            self.b = np.random.randn(Sout,1)/np.sqrt(Sin)
            self.inV = None

        def forward(self,inV):
            outV = np.matmul(np.transpose(self.W),inV)+ self.b
            self.inV = inV  #storing the input vector for backpropagation
            return outV

        def backprop(self,grad):
            Lgrad = np.matmul(self.W,grad)  #multiply the input grad  with the gradient wrt input which is the weight matrix 
            self.W = self.W - 0.005*np.matmul(self.inV,np.transpose(grad)) #updating the weights and biases with their respective gradients 
            self.b = self.b - 0.005*grad            
            return Lgrad

class Relu:
        def __init__(self):
            self.inV = None

        def forward(self,inV):
            outV = np.zeros((len(inV),1))
            for i in range(0,len(outV)):
                outV[i][0] = max(0,inV[i][0])
            self.inV =inV
            return outV

        def backprop(self,grad):
            grad[self.inV<0]= 0;
            return grad

class Softmax:
        def __init__(self):
            self.outV = None

        def forward(self,inV):
            outV = np.exp(inV)
            outV = outV/np.sum(outV)
            self.outV = outV
            return outV

        def backprop(self,grad):
            size = self.outV.shape[0]
            Lgrad = np.zeros((size,size))
            for i in range(size):
                for j in range(size):
                    if(i==j):
                        Lgrad[i][i] = self.outV[i][0]*(1-self.outV[i][0]) 
                    else:
                        Lgrad[i][j] = -self.outV[i][0]*self.outV[j][0]
                        Lgrad[j][i] = -self.outV[i][0]*self.outV[j][0]
            grad = np.matmul(Lgrad,grad)
            return grad
class CEloss:
        def __init__(self):
            self.label = None
            self.inV = None

        def forward(self,inV,label):
            Loss = -np.log(inV[label][0])
            self.label = label
            self.inV = inV
            return Loss

        def backprop(self,grad):
            grad = np.zeros((self.inV.shape[0],1))
            grad[self.label][0] = -1/self.inV[self.label][0]
            return grad

class neuralnet:
        def __init__(self):
            self.Layers = np.array([])

        def forward(self,inV,label):
            outV = inV
            for i in range(0,len(self.Layers)-1):
                outV = self.Layers[i].forward(outV)
            outV = self.Layers[len(self.Layers)-1].forward(outV,label)
            return outV

        def backprop(self):
            grad = np.array([])
            for i in range(len(self.Layers)-1,-1,-1):
                grad = self.Layers[i].backprop(grad)
               # print(grad.shape)
            return

        def addlayer(self,layer):
            self.Layers = np.append(self.Layers,layer)


In [ ]:
NN = neuralnet()
NN.addlayer(FClayer(3072,400))
NN.addlayer(Relu())
NN.addlayer(FClayer(400,50))
NN.addlayer(Relu())
NN.addlayer(FClayer(50,2))
NN.addlayer(Softmax())
NN.addlayer(CEloss())

test_count = 0
train_count = 0
test_loss = 0
train_loss = 0
lossarray = np.array([])
testlossarray = np.array([])

for epoch in range(5):
    for image,label in trainloader:
        if label<2:
            image = (image.view(-1,1)).numpy()

            train_loss += NN.forward(image,label)
            NN.backprop()

            train_count += 1

            if train_count%1000==0:
                correct = 0
                test_count = 0

                for image, label in testloader:
                    if label<2:
                        image = (image.view(-1,1)).numpy()

                        test_loss += NN.forward(image,label)

                        prob = NN.Layers[5].outV
                        top_class = np.argmax(prob)
                        if label == top_class:
                            correct += 1

                        test_count += 1

                print(f"Epoch {int(train_count/500)}.." f"Train_loss: {train_loss/500:.3f}.." f"Test_loss: {test_loss/test_count:.3f}.."
                      f"Accuracy: {correct/test_count:.3f}..")

                lossarray = np.append(lossarray,train_loss/500)
                testlossarray = np.append(testlossarray,test_loss/test_count)
                test_loss = 0
                train_loss = 0

plt.plot(lossarray, label='Training loss')
plt.plot(testlossarray, label='Testing loss')
plt.legend(frameon=False)
plt.savefig('tr_2.png', dpi=200)
plt.show()
